In [352]:
import pymysql
import pandas as pd
import numpy as np
from ast import literal_eval
db=pymysql.connect(
    user='root',
    passwd='111111',
    host='localhost',
    db='instagram',
    charset='utf8'
)

cursor=db.cursor(pymysql.cursors.DictCursor)

sql="select * from user_post;"
cursor.execute(sql)
result=cursor.fetchall() #DB에서 user_post 받아오기

user_post_df=pd.DataFrame(result) #받아 온 값을 dataframe형식으로 변환 
user_post_df.head(1)

,id,post_index,follow,follower,bookmark_post
0,0000,None,"1111,2222,3333","1111,2222,3333,4444",None


In [353]:
user_post_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             10 non-null     object
 1   post_index     0 non-null      object
 2   follow         10 non-null     object
 3   follower       10 non-null     object
 4   bookmark_post  0 non-null      object
dtypes: object(5)
memory usage: 528.0+ bytes


In [354]:
print(user_post_df['follow'])

0                   1111,2222,3333
1                        2222,5555
2              0000,1111,6666,7777
3              1111,4444,2222,6666
4              3333,0000,7777,6666
5    2222,4444,7777,9999,3333,8888
6                   2222,4444,8888
7                        5555,6666
8                   4444,5555,7777
9                             0000
Name: follow, dtype: object


In [355]:
user_post_df['follow']=user_post_df['follow'].str.split(',') # ','로 나눔
print(user_post_df['follow'])

0                      [1111, 2222, 3333]
1                            [2222, 5555]
2                [0000, 1111, 6666, 7777]
3                [1111, 4444, 2222, 6666]
4                [3333, 0000, 7777, 6666]
5    [2222, 4444, 7777, 9999, 3333, 8888]
6                      [2222, 4444, 8888]
7                            [5555, 6666]
8                      [4444, 5555, 7777]
9                                  [0000]
Name: follow, dtype: object


In [356]:
user_post_df[['follow']][:1]

,follow
0,"[1111, 2222, 3333]"


In [357]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환.
user_post_df['follow_literal']=user_post_df['follow'].apply(lambda x:(' ').join(x))
print(user_post_df['follow_literal'])

#follow 문자열을 Count 백터화(나눈 문자의 출현 빈도를 세어 벡터(집함)로 변환)
count_vect=CountVectorizer(min_df=0,ngram_range=(1,2))
follow_mat=count_vect.fit_transform(user_post_df['follow_literal'])
print(follow_mat.shape)

0                   1111 2222 3333
1                        2222 5555
2              0000 1111 6666 7777
3              1111 4444 2222 6666
4              3333 0000 7777 6666
5    2222 4444 7777 9999 3333 8888
6                   2222 4444 8888
7                        5555 6666
8                   4444 5555 7777
9                             0000
Name: follow_literal, dtype: object
(10, 31)


In [358]:
from sklearn.metrics.pairwise import cosine_similarity
#각 id에 따른 팔로우별 cosine유사도 추출
follow_sim=cosine_similarity(follow_mat,follow_mat)
print(follow_sim.shape)

(10, 10)


In [359]:
follow_sim_sorted_ind=follow_sim.argsort()[:,::-1]#index로 정렬 (::-1은 전체를 역순으로 조회)
print(follow_sim_sorted_ind)

[[0 3 5 1 6 4 2 9 8 7]
 [1 7 8 6 0 3 5 9 4 2]
 [2 4 9 3 7 8 0 5 6 1]
 [3 6 0 2 5 7 1 8 4 9]
 [4 2 9 5 7 8 0 3 6 1]
 [5 6 8 0 4 3 1 2 9 7]
 [6 5 3 1 8 0 9 7 4 2]
 [7 1 8 4 3 2 9 6 5 0]
 [8 5 7 1 6 4 3 2 9 0]
 [9 4 2 8 7 6 5 3 1 0]]


In [360]:
#유사도가 높은 id를 찾아 반환
def find_sim_follow(df,sorted_ind,id,top_n=6):
    user_id=df[df['id']==id]
    
    id_index=user_id.index.values
    result_ind=[[]]
    #선택된 아이디의 팔로우한 아이디를 찾아 아닌 것 리스트에 저장
    for x in df['id']:
        for y in user_id['follow']:
            if x in y:
                continue
            else:
                result_ind[0].append(x)
        if len(result_ind)==5:
                    break
    result_ind=np.array(result_ind) 
    similar_indexes=sorted_ind[id_index,:(top_n)] #6개 저장
    similar_indexes=np.delete(similar_indexes,[0],axis=1) #자기자신은 삭제
    similar_indexes=similar_indexes.reshape(-1)#해당 열(index)만 따로 저장
    
    similar_df=df.iloc[similar_indexes]#해당 index에 대한 아이디를 저장
    count_list=[]
    user_id_list=user_id['follow'].tolist()
    similar_df_list=similar_df['follow'].tolist()
    print(similar_df_list)
    i=0
    #함께아는 친구 계산
    for lenght in range(len(similar_df_list)):#5번(추천된 5명의 count)
        count=0
        for x in similar_df_list[i]:#추천된 사람을 한 명씩 호출해 그 사람이 팔로우 한 사람을 찾음
            for y in user_id_list[0]:#자신이 팔로우한 사람
                if x in y:#자신이 팔로우한 사람과 추천된 사람의 팔로우를 비교
                    count=count+1
        i=i+1
        count_list.append(count)
                    
    similar_df['together']=count_list
    return similar_df

In [361]:
similar_id=find_sim_follow(user_post_df,follow_sim_sorted_ind,'9999')
similar_id[['id','follow','together']]

[['3333', '0000', '7777', '6666'], ['0000', '1111', '6666', '7777'], ['4444', '5555', '7777'], ['5555', '6666'], ['2222', '4444', '8888']]


<ipython-input-360-5ce9b42d7a20>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similar_df['together']=count_list


,id,follow,together
4,4444,"[3333, 0000, 7777, 6666]",1
2,2222,"[0000, 1111, 6666, 7777]",1
8,8888,"[4444, 5555, 7777]",0
7,7777,"[5555, 6666]",0
6,6666,"[2222, 4444, 8888]",0


,id,follow,togeter
7,7777,"[5555, 6666]",1
8,8888,"[4444, 5555, 7777]",1
6,6666,"[2222, 4444, 8888]",1
0,0000,"[1111, 2222, 3333]",1
3,3333,"[1111, 4444, 2222, 6666]",1
